Data Wrangling Notebook for SY Data
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [41]:
import pandas as pd
from pandas import Series
import numpy as np
import re
import uuid 

Silencing warnings that are unnecessary

In [42]:
try:
    import warnings
    warnings.filterwarnings('ignore')
except:
    pass

Import SY Data

In [43]:
sy_data = pd.read_csv("./../Original_Data/futres_sy_mod.csv")

Rename Columns

In [44]:
sy_data = sy_data.rename(columns={'ID': 'individualID',
                                  'Taxa':'scientificName',
                                  'Measurements (NAME;NAME;NAME...) divided by ;':'measurementType',
                                  'Values (mm;mm;mm...) divided by ;':'measurementValue',
                                  'Gender': 'sex',
                                  'Age appearance':'lifeStage',
                                  'Siding':'measurementSide'})

Clean up lifeStage column

In [45]:
# Create Filters
adult = sy_data['lifeStage']=="ADL"
juvenile = sy_data['lifeStage']=="JUV"

# Assign correct terms using filters
sy_data['lifeStage'][adult] = "adult"
sy_data['lifeStage'][juvenile] = "juvenile"

Clean up sex column

In [46]:
male = sy_data['sex']=="M"
sy_data["sex"][male==True]="male"
sy_data['sex'][male==False]=""

Clean up side column

In [47]:
right = sy_data['measurementSide'] == "IZQ"
left = sy_data['measurementSide'] == "DER"
sy_data['measurementSide'][(right == False) & (left == False)] = ""
sy_data['measurementSide'][right == True] = "right"
sy_data['measurementSide'][left == True] = "left"

Add samplingProtocol and set it to reference values

In [48]:
sy_data = sy_data.assign(samplingProtocol = sy_data["PROTOCOL"])

Check is length of measurementType and measurementValue are the same 

In [49]:
#def matching_size(row):
#    """function to match size of measurement type and value"""
#    mtype = row["measurementType"].split(';')
#    mval = row["measurementValue"].split(';')
#    if len(mtype) != len(mval):
#        print ("Error: Size Mismatch")
#        print(row.name)
        
#sy_data.apply(matching_size,  axis =1)

JSON specific columns and add to dynamicProperties

In [50]:
# Specify columns for dynamic properties
json_columns = sy_data.iloc[:,19:45]

# Create dynamicProperties
sy_data = sy_data.assign(dynamicProperties="")

for i in json_columns.index:
    sy_data["dynamicProperties"][i] = json_columns.loc[i].to_json()

Rename specimens and append Specimen to measurementType

In [51]:
# Rename Specimens (useful later down the line)
sy_data["Specimen"] = sy_data["Specimen"].replace({'metacarpo': 'metacarpal', 
                                                   'falange 1º': 'proximal phalanx',
                                                   'metatarso': 'metatarsal', 
                                                   'coxal':'os coxae'})

# Append Specimen name to measurementType Column
sy_data["measurementType"] = sy_data["Specimen"] + sy_data["measurementType"]

# Replace names of terms avaliable in GEOME
sy_data["measurementType"] = sy_data["measurementType"].replace({'femurGL': 'femur length'})

0           metacarpalBd
1           metacarpalDd
2     proximal phalanxGL
3     proximal phalanxBp
4     proximal phalanxBd
             ...        
64               femurBd
65            os coxaeLA
66           os coxaeLAR
67            os coxaeSH
68            os coxaeSB
Name: measurementType, Length: 69, dtype: object

Subsetting by template terms

In [12]:
#Create column list
cols = sy_data.columns.tolist()

#Specify desired columns
cols = ['individualID',
        'scientificName',
        'measurementType',
        'measurementValue',
        'sex',
        'lifeStage',
        'dynamicProperties',
        'samplingProtocol']

#Subset dataframe
sy_data = sy_data[cols]

Subset by avaliable measurement terms

In [14]:
sy_data = sy_data[sy_data['measurementType']=="femur length"]

Drop Rows that contain NA in measurementValue

In [ ]:
sy_data = sy_data.dropna(subset=['measurementValue'])

Create necessary materialSampleID column and populate with UUID (use hex to remove dashes). Create necessary eventID column and populate with materialSampleID column

In [52]:
sy_data = sy_data.assign(materialSampleID = '')
sy_data['materialSampleID'] = [uuid.uuid4().hex for _ in range(len(sy_data.index))]

sy_data = sy_data.assign(eventID = sy_data["materialSampleID"])

GEOME requires certain columns. Columns created and if unknown assign "unknown"

In [53]:
sy_data = sy_data.assign(basisOfRecord="Unknown")
sy_data = sy_data.assign(locality="Unknown")
sy_data = sy_data.assign(yearCollected="Unknown")
sy_data = sy_data.assign(measurementMethod="Unknown")

Assign diagnosticID with unique number

In [55]:
# Create diagnosticID which is a unique number for each measurement
sy_data = sy_data.assign(diagnosticID = '')
sy_data['diagnosticID'] = np.arange(len(sy_data))

Write file as csv for GEOME upload

In [ ]:
# Writing long data csv file
sy_data.to_csv('../Mapped_Data/sy_data.csv', index=False)